**Import Packages**

In [72]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras as tf
from keras.layers import Dense, Activation
from keras import Sequential
from keras.models import load_model as tl
from keras.optimizers import Adam

**Read dataset**

In [73]:
data = pd.read_csv("/content/drive/MyDrive/Dataset_CKD.csv")
print(data)

      id   age    bp     sg   al   su     rbc        pc         pcc  \
0      0  48.0  80.0  1.020  1.0  0.0     NaN    normal  notpresent   
1      1   7.0  50.0  1.020  4.0  0.0     NaN    normal  notpresent   
2      2  62.0  80.0  1.010  2.0  3.0  normal    normal  notpresent   
3      3  48.0  70.0  1.005  4.0  0.0  normal  abnormal     present   
4      4  51.0  80.0  1.010  2.0  0.0  normal    normal  notpresent   
..   ...   ...   ...    ...  ...  ...     ...       ...         ...   
395  395  55.0  80.0  1.020  0.0  0.0  normal    normal  notpresent   
396  396  42.0  70.0  1.025  0.0  0.0  normal    normal  notpresent   
397  397  12.0  80.0  1.020  0.0  0.0  normal    normal  notpresent   
398  398  17.0  60.0  1.025  0.0  0.0  normal    normal  notpresent   
399  399  58.0  80.0  1.025  0.0  0.0  normal    normal  notpresent   

             ba  ...  pcv    wc   rc  htn   dm  cad appet   pe  ane  \
0    notpresent  ...   44  7800  5.2  yes  yes   no  good   no   no   
1    

**Understanding Data Type and Features**

In [74]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              400 non-null    int64  
 1   age             391 non-null    float64
 2   bp              388 non-null    float64
 3   sg              353 non-null    float64
 4   al              354 non-null    float64
 5   su              351 non-null    float64
 6   rbc             248 non-null    object 
 7   pc              335 non-null    object 
 8   pcc             396 non-null    object 
 9   ba              396 non-null    object 
 10  bgr             356 non-null    float64
 11  bu              381 non-null    float64
 12  sc              383 non-null    float64
 13  sod             313 non-null    float64
 14  pot             312 non-null    float64
 15  hemo            348 non-null    float64
 16  pcv             330 non-null    object 
 17  wc              295 non-null    obj

In [75]:
data.drop('id', inplace=True, axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             391 non-null    float64
 1   bp              388 non-null    float64
 2   sg              353 non-null    float64
 3   al              354 non-null    float64
 4   su              351 non-null    float64
 5   rbc             248 non-null    object 
 6   pc              335 non-null    object 
 7   pcc             396 non-null    object 
 8   ba              396 non-null    object 
 9   bgr             356 non-null    float64
 10  bu              381 non-null    float64
 11  sc              383 non-null    float64
 12  sod             313 non-null    float64
 13  pot             312 non-null    float64
 14  hemo            348 non-null    float64
 15  pcv             330 non-null    object 
 16  wc              295 non-null    object 
 17  rc              270 non-null    obj

**Handling Missing Values**

Remove null values

In [76]:
data=data.dropna(how="any")
print(data)

      age    bp     sg   al   su       rbc        pc         pcc          ba  \
3    48.0  70.0  1.005  4.0  0.0    normal  abnormal     present  notpresent   
9    53.0  90.0  1.020  2.0  0.0  abnormal  abnormal     present  notpresent   
11   63.0  70.0  1.010  3.0  0.0  abnormal  abnormal     present  notpresent   
14   68.0  80.0  1.010  3.0  2.0    normal  abnormal     present     present   
20   61.0  80.0  1.015  2.0  0.0  abnormal  abnormal  notpresent  notpresent   
..    ...   ...    ...  ...  ...       ...       ...         ...         ...   
395  55.0  80.0  1.020  0.0  0.0    normal    normal  notpresent  notpresent   
396  42.0  70.0  1.025  0.0  0.0    normal    normal  notpresent  notpresent   
397  12.0  80.0  1.020  0.0  0.0    normal    normal  notpresent  notpresent   
398  17.0  60.0  1.025  0.0  0.0    normal    normal  notpresent  notpresent   
399  58.0  80.0  1.025  0.0  0.0    normal    normal  notpresent  notpresent   

       bgr  ...  pcv     wc   rc  htn  

**Label Encoding** (String values to Numeric values)

In [77]:
data['rbc'] = data['rbc'].map({"abnormal":1,"normal":0})
data['pc'] = data['pc'].map({"abnormal":1,"normal":0})
data['pcc'] = data['pcc'].map({"present":1,"notpresent":0})
data['ba'] = data['ba'].map({"present":1,"notpresent":0})
data['htn'] = data['htn'].map({"yes":1,"no":0})
data['dm'] = data['dm'].map({"yes":1,"no":0})
data['cad'] = data['cad'].map({"yes":1,"no":0})
data['pe'] = data['pe'].map({"yes":1,"no":0})
data['ane'] = data['ane'].map({"yes":1,"no":0})
data['appet'] = data['appet'].map({"poor":1,"good":0})
data['classification'] = data['classification'].map({"ckd":1,"notckd":0})
data['pcv'] = data['pcv'].astype('int')
data['wc'] = data['wc'].astype('int')
data['rc'] = data['rc'].astype('float')
print(data)

      age    bp     sg   al   su  rbc  pc  pcc  ba    bgr  ...  pcv     wc  \
3    48.0  70.0  1.005  4.0  0.0    0   1    1   0  117.0  ...   32   6700   
9    53.0  90.0  1.020  2.0  0.0    1   1    1   0   70.0  ...   29  12100   
11   63.0  70.0  1.010  3.0  0.0    1   1    1   0  380.0  ...   32   4500   
14   68.0  80.0  1.010  3.0  2.0    0   1    1   1  157.0  ...   16  11000   
20   61.0  80.0  1.015  2.0  0.0    1   1    0   0  173.0  ...   24   9200   
..    ...   ...    ...  ...  ...  ...  ..  ...  ..    ...  ...  ...    ...   
395  55.0  80.0  1.020  0.0  0.0    0   0    0   0  140.0  ...   47   6700   
396  42.0  70.0  1.025  0.0  0.0    0   0    0   0   75.0  ...   54   7800   
397  12.0  80.0  1.020  0.0  0.0    0   0    0   0  100.0  ...   49   6600   
398  17.0  60.0  1.025  0.0  0.0    0   0    0   0  114.0  ...   51   7200   
399  58.0  80.0  1.025  0.0  0.0    0   0    0   0  131.0  ...   53   6800   

      rc  htn  dm  cad  appet  pe  ane  classification  
3    3

**Splitting Dependent and Independent Variable**

In [78]:
X = data.iloc[:,1:24].values
y = data.iloc[:, 24].values

**Split Train and Test set**

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30,
random_state = 121)#101
print("X train value")
print(X_train)
print("Y train value")
print(y_train)

X train value
[[70.     1.02   0.    ...  0.     0.     0.   ]
 [60.     1.02   0.    ...  0.     0.     0.   ]
 [70.     1.025  0.    ...  0.     0.     0.   ]
 ...
 [80.     1.02   0.    ...  0.     0.     0.   ]
 [80.     1.02   2.    ...  0.     0.     0.   ]
 [60.     1.025  0.    ...  0.     0.     0.   ]]
Y train value
[0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0
 0 1 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0]


**Model Building**

In [80]:
model = tf.Sequential() 
model.add(tf.layers.Dense(64,input_dim=23,activation='relu'))

**Hidden and Output Layer**

In [81]:
model.add(tf.layers.Dense(128,activation='relu')) 
model.add(tf.layers.Dense(256,activation='relu')) 
model.add(tf.layers.Dense(512,activation='relu')) 
model.add(tf.layers.Dense(1,activation='sigmoid')) 

**Compile Model**

In [82]:
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=1000)

Epoch 1/1000
4/4 [==============================] - 1s 13ms/step - loss: 250.4937 - accuracy: 0.4000
Epoch 2/1000
4/4 [==============================] - 0s 8ms/step - loss: 127.8007 - accuracy: 0.6909
Epoch 3/1000
4/4 [==============================] - 0s 8ms/step - loss: 44.7311 - accuracy: 0.5455
Epoch 4/1000
4/4 [==============================] - 0s 8ms/step - loss: 25.9030 - accuracy: 0.6182
Epoch 5/1000
4/4 [==============================] - 0s 7ms/step - loss: 27.8122 - accuracy: 0.5273
Epoch 6/1000
4/4 [==============================] - 0s 7ms/step - loss: 19.9468 - accuracy: 0.5818
Epoch 7/1000
4/4 [==============================] - 0s 8ms/step - loss: 15.3649 - accuracy: 0.4909
Epoch 8/1000
4/4 [==============================] - 0s 7ms/step - loss: 13.0705 - accuracy: 0.5455
Epoch 9/1000
4/4 [==============================] - 0s 8ms/step - loss: 14.5122 - accuracy: 0.5273
Epoch 10/1000
4/4 [==============================] - 0s 7ms/step - loss: 20.4153 - accuracy: 0.4182
Epoch 

**Test**

In [83]:
ypred=model.predict(X_test)
ypred = ypred.round()
print(ypred)
print(y_test)

2/2 [==============================] - 0s 9ms/step
[[0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]]
[0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0
 1 0 0 1 0 0 0 0 0 1 1]


**Model Evaluation**

In [84]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,ypred)
print(cm)

[[39  0]
 [ 0  9]]


**Saved Model**

In [ ]:
import pickle
file = open('/content/drive/MyDrive/ckd.pkl','wb')
pickle.dump(model,file)

In [ ]:
from google.colab import files
files.download('/content/drive/MyDrive/ckd.pkl')